# FUNCTIONS FROM ORIGINAL SCRIPT

In [1]:
import numpy as np
import pandas as pd
import astropy.io.fits as apfits
import astropy.visualization as apvis
import astropy.nddata as apnndd
import astropy.table as aptable
import astropy.wcs as apwcs
import astropy.units as apunits
import astropy.coordinates as apcoords
import matplotlib.pyplot as mplplot
import matplotlib.patches as mplpatches
import glob
import os
import scipy as sp
import scipy.optimize as spopt
import copy
import grismconf
import sys
import spectres

# Accessing Fengwu Data Files (OneDrive)

## Load in frame file and designate variables of the filter, grism and module of the input frame file from the header

Comments on file:
- Each file contains multiple headers! SEE THESE THROUGH print(Frame) THEN CHECK INDIVIDUALLY THROUGH Frame[0] or ['PRIMARY'] 
- To see the name of headers past ['PRIMARY'], see "EXTNAME" in their header
- Header has all the secrets (as per)
- I believe the reason we cannot drizzlepac the frames together is that the file type (Frame['PRIMARY'].header['FILETYPE']) = 'raw'
- When it comes to using drizzlepac, the "Dither Information" keywords will be essential in combining the correct files
- File name origins outlined in header "Observation identifiers" keywords
- CDRS Information at bottom header must be files set in MIRAGE - ask Fengwu?


In [2]:
# F322W2 R A - trace cross-dispersion shift = -25.6906816 to -22.448626
# WORKS
# file_name = "/Users/zx446701/OneDrive - The Open University/Ice_Age/Fengwu_Sims_2/F322W2_grism/F322W2_Level15_Frames/jw01309023002_01101_00018_nrca5_rate_lv1.5.fits"

# F322W2 R B - trace cross-dispersion shift = 9.28123335 to 6.83300615
# WORKS
# file_name = "/Users/zx446701/OneDrive - The Open University/Ice_Age/Fengwu_Sims_2/F322W2_grism/F322W2_Level15_Frames/jw01309023002_01101_00018_nrcb5_rate_lv1.5.fits"

# F322W2 C A - trace cross-dispersion shift =
# WORKS
# file_name = "/Users/zx446701/OneDrive - The Open University/Ice_Age/Fengwu_Sims_2/F322W2_grism/F322W2_Level15_Frames/jw01309024001_01101_00007_nrca5_rate_lv1.5.fits"

# F322W2 C B - trace cross-dispersion shift =
# WORKS
# file_name = "/Users/zx446701/OneDrive - The Open University/Ice_Age/Fengwu_Sims_2/F322W2_grism/F322W2_Level15_Frames/jw01309024001_01101_00007_nrcb5_rate_lv1.5.fits"

# F444W R A - Trace shift (-20 to -23 from direct coords)
# WORKS
file_name = "/Users/zx446701/OneDrive - The Open University/Ice_Age/Fengwu_Sims_2/F444W_grism/F444W_Level15_Frames/jw01309025001_01101_00002_nrca5_rate_lv1.5.fits"

# F444W R B - trace cross-dispersion shift =
# WORKS
# file_name = "/Users/zx446701/OneDrive - The Open University/Ice_Age/Fengwu_Sims_2/F444W_grism/F444W_Level15_Frames/jw01309025001_01101_00002_nrcb5_rate_lv1.5.fits"

# F444W C A - trace cross-dispersion shift =
# WORKS
# file_name = "/Users/zx446701/OneDrive - The Open University/Ice_Age/Fengwu_Sims_2/F444W_grism/F444W_Level15_Frames/jw01309026001_01101_00007_nrca5_rate_lv1.5.fits"

# F444W C B - trace cross-dispersion shift = 
# 
# file_name = "/Users/zx446701/OneDrive - The Open University/Ice_Age/Fengwu_Sims_2/F444W_grism/F444W_Level15_Frames/jw01309026001_01101_00007_nrcb5_rate_lv1.5.fits"

fits_file = apfits.open(file_name)
data = fits_file["SCI"].data

# print(repr(fits_file["PRIMARY"].header))
# print(repr(fits_file["SCI"].header))
filter_ = fits_file['PRIMARY'].header['FILTER'].strip()
# [-1] needed to extract just either R or C
# CHANGE DIRECTION TO GRISM FOR CLARITY
direction = fits_file['PRIMARY'].header['PUPIL'].strip()[-1]
module = fits_file['PRIMARY'].header['MODULE'].strip()
dither = fits_file['PRIMARY'].header['PATT_NUM']
exposure = fits_file['PRIMARY'].header['EXPOSURE']
# SUBPIXNUM Missing from new sims!!!
# subpixel = Frame['PRIMARY'].header['SUBPXNUM']
# print(filter_,direction,module)

## Create variables for the folders needed throughout script

- Will be used for file path joining

In [3]:
# All 192 simulated frames produced by MIRAGE - total = 192 frames as 4 observations each have 2 Grisms, 2 modules, 3 Primary Dithers each with 4 subpixel dithers.
# Of the 4 observations 2 are with the F322W2 (Water) Filter and F444W (CO+CO2 filter), making 96 frames per filter. This is how the folders are split.
frameFitsDir = f"/Users/zx446701/OneDrive - The Open University/Ice_Age/Fengwu_Sims_2/{filter_}_grism/{filter_}_Level15_Frames/"

# Source list for every frame within the simulation - WANT TO CREATE OUR OWN SOURCE FROM DIRECT IMAGES
frameListDir = f"/Users/zx446701/OneDrive - The Open University/Ice_Age/Fengwu_Sims_2/{filter_}_grism/{filter_}_List"


## Import PSF model grid modules

In [4]:
# Use source code to look into utils etc.
from webbpsf.utils import to_griddedpsfmodel
from webbpsf.gridded_library import display_psf_grid
# Take specific instrument, module, filter and detector PSF fits files 
# and turn into a grid of how the PSF changes with position on detector array
# NUMBER OF DETECTOR IS IN NRCA5 - 5 IS LW DETECTOR 1-4 IS SW
# ONLY 5 REQUIRED FOR WFSS AS ONLY DETECTOR ABLE TO DO THIS
if filter_ == 'F322W2':
    grid = to_griddedpsfmodel(
        f"/Users/zx446701/OneDrive - The Open University/Ice_Age/Fengwu Data/gridded_psf_library/nircam_nrc{module.lower()}5_{filter_.lower()}_clear_fovp47_samp5_npsf16_requirements_realization0.fits"
    )
else:
    grid = to_griddedpsfmodel(
    f"/Users/zx446701/OneDrive - The Open University/Ice_Age/Fengwu Data/gridded_psf_library/nircam_nrc{module.lower()}5_{filter_.lower()}_clear_fovp61_samp5_npsf16_requirements_realization0.fits"
    )
    

/Users/zx446701/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/pysynphot/locations.py:46: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/Users/zx446701/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


**WARNING**: LOCAL JWST PRD VERSION PRDOPSSOC-040-001 DOESN'T MATCH THE CURRENT ONLINE VERSION PRDOPSSOC-045-003
Please consider updating pysiaf, e.g. pip install --upgrade pysiaf or conda update pysiaf


# Define functions to extract the source list, along with sky and pixel coordinates within the images


## Function creates pathname of input frame file and opens as a pandas dataframe 
Can I find the information for what each column represents in this file?

10 columns total: 

Col 0: Source ID ??

Col 1-4: Coords (RA and Dec in hms, dms then deg, deg)

Col 5-6: Pixel Coordinates

Col 7-10: No idea

- Extracts name of the file from the full pathname
        print(os.path.basename(file_name))
- Creates list of each component of the file name SPLIT by _ (each as a string) and selects the first 4 components
        print(os.path.basename(file_name).split('_')[:4])
- Rejoins the 4 extracted components by JOINing them with _
        print('_'.join(os.path.basename(file_name).split('_')[:4]))

In [5]:
def getSourceListForImage(image, frameListDir):
    listPath = os.path.join(
        frameListDir,
        f"{'_'.join(os.path.basename(image).split('_')[:4])}_uncal_pointsources.list",
    )
    return pd.read_csv(listPath, delim_whitespace=True, comment="#", header=None)

## Function extracting Sky Coordinates from source list

- Identify all the values within col 3 and 4 of pandas df
        sourceList.loc[:, [3, 4]]
- Turns this into a numpy array
        sourceList.loc[:, [3, 4]].to_numpy()
- Finally tranpose the the 2D array to a list of RA values and Dec values
        sourceList.loc[:, [3, 4]].to_numpy().T
- Starred list indicates 0 and 1 lists within of 2d array ([[0],[1]]) inputted here so every sources RA and Dec can be inputted and coverted into astropy SkyCoord form
        apcoords.SkyCoord(*sourceList.loc[:, [3, 4]].to_numpy().T, frame=apcoords.ICRS, unit=apunits.deg)


In [6]:
def getSourceCoordsForImage(image, frameListDir):
    sourceList = getSourceListForImage(image, frameListDir) # Previous Function
    coords = apcoords.SkyCoord(
        *sourceList.loc[:, [3, 4]].to_numpy().T, frame=apcoords.ICRS, unit=apunits.deg
    )
    return coords

## Function extracting Pixel Coords from source list

- Does the same as above except it does not need to transpose values as no additional conversion function needed

In [7]:
def getSourcePixelsForImage(image, frameListDir):
    sourceList = getSourceListForImage(image, frameListDir)
    pixels = sourceList.loc[:, [5, 6]].to_numpy()
    
    return pixels


# Plotting Functions

### This function is not used

Just calculates the length and width of a trace from a given x,y pixel position 

In [8]:
def computeTrace(pixels, fac=100, filter_="F322W2", module="A", direction="R", simYDisp=False, order=1):
    # Locate config File for the module and grism direction 
    confFile = f"/Users/zx446701/OneDrive - The Open University/Ice_Age/Fengwu_Sims_2/GRISM_NIRCAM-master/V3/NIRCAM_{filter_}_mod{module}_{direction}.conf"
    # Class to read and hold GRISM configuration info
    conf = grismconf.Config(confFile)
    # Found from GRISMCONF README file - see link above
    # Middle section - number of pixels from end to start in X direction
    # 1/ middle = slighting trace by number of pixels
    # /fac is for splitting by subpixel amounts and oversampling
    dt = np.abs(1 / (1 + conf.DISPX(f'+{order}', *pixels, 1) - conf.DISPX(f'+{order}', *pixels, 0)) / fac)
    # t is the trace and how much of it is covered (0 to 1 is the full trace)
    t = np.arange(0, 1, dt)

    # DISP(X,Y,L) = DISPERSION POLYNOMIAL (X direction, Y, Full Length)
    # order, x0, y0, steps along dispersion between 0 and 1
    # X disp polynomial
    dxs = conf.DISPX(f"+{order}" *pixels, t)
    # Y disp polynomial 
    dys = conf.DISPY(f"+{order}" *pixels, t)
    # Compute wavelength of each pixel
    wavs = conf.DISPL(f"+{order}" *pixels, t)

    return (
        pixels[0] + dxs,
        pixels[1] + dys if simYDisp else np.full_like(dys, pixels[1]),
        wavs,
    )

### Function returning the pixels and wavelengths of those pixels of a source dispersed in the R direction


In [9]:
def computeTraceWLR(pixels, filter_="F322W2", module="A", simYDisp=False, order=1):
    # Locate config File for the filter, module and grism direction 
    confFile = f"/Users/zx446701/OneDrive - The Open University/Ice_Age/Fengwu_Sims_2/GRISM_NIRCAM-master/V3/NIRCAM_{filter_}_mod{module}_R.conf"
    # Class to read and hold GRISM configuration info
    conf = grismconf.Config(confFile)

    # dt = 1/(conf.DISPX(f'+{order}', *pixels,0)-conf.DISPX(f'+{order}', *pixels,1)) / fac
    # t = np.arange(0,1,dt)

    # 
    dxs0 = conf.DISPX(f'+{order}',*pixels, 0)

    #
    dxs1 = conf.DISPX(f'+{order}', *pixels, 1)
    
    # np.floor rounds to lower value
    # np.ceil rounds to higher value
    # Want this so full trace is picked up in all pixels
    if module == "A":
    # FOR MODULE A IN DIRECTION R
    # +1 REQUIRED DUE TO ARANGE NEEDING TO INCLUDE LAST TRACE PIXEL NUMBER
        dxs = np.arange(np.floor(dxs0),np.ceil(dxs1)+1).astype(int)

    else:
        # if dxs is decreasing then switch floor and ceil
        dxs = np.arange(np.floor(dxs1),np.ceil(dxs0)+1).astype(int)
    
    ts = conf.INVDISPX(f'+{order}',*pixels,dxs)
    
    dys = conf.DISPY(f'+{order}',*pixels,ts)

    wavs = conf.DISPL(f'+{order}',*pixels,ts)
    
    return (
        pixels[0] + dxs,
        pixels[1] + dys,
        wavs,
    )

### Function returning the pixels and wavelengths of those pixels of a source dispersed in the C direction


In [10]:
def computeTraceWLC(
    pixels,
    filter_="F322W2",
    module="A",
    simYDisp=False,
    order=1
):
    
    # Locate config File for the filter, module and grism direction 
    confFile = f"/Users/zx446701/OneDrive - The Open University/Ice_Age/Fengwu_Sims_2/GRISM_NIRCAM-master/V3/NIRCAM_{filter_}_mod{module}_C.conf"
    # Class to read and hold GRISM configuration info
    conf = grismconf.Config(confFile)
    # Found from GRISMCONF README file - see link above


    # dt = 1/(conf.DISPX(f'+{order}', *pixels,0)-conf.DISPX(f'+{order}', *pixels,1)) / fac
    # t = np.arange(0,1,dt)

    dys0 = conf.DISPY(f'+{order}',*pixels, 0)

    dys1 = conf.DISPY(f'+{order}', *pixels, 1)
    
    # np.floor rounds to lower value
    # np.ceil rounds to higher value
    # Want this so full trace is picked up in all pixels
    
    dys = np.arange(np.floor(dys0),np.ceil(dys1)+1).astype(int)

    ts = conf.INVDISPY(f'+{order}',*pixels,dys)
    
    dxs = conf.DISPX(f'+{order}',*pixels,ts)

    wavs = conf.DISPL(f'+{order}',*pixels,ts)

    
    return (
        pixels[0] + dxs,
        pixels[1] + dys,
        wavs,
    )

### Function to choose between R or C wavelengths required

In [11]:
def computeTraceWL(
    pixels,
    filter_="F322W2",
    module="A",
    direction="R",
    simYDisp=False,
    order=1
):
    if direction == "R":
        return computeTraceWLR(pixels=pixels,filter_=filter_,module=module,simYDisp=simYDisp,order=order)
    else:
        return computeTraceWLC(pixels=pixels,filter_=filter_,module=module,simYDisp=simYDisp,order=order)

### Function to calculate the trace box of a source dispersed in the R direction

In [12]:
def computeTraceBoxR(
    # Pixels of source being traced (x0, y0)
    pixels,
    # Not sure this is needed
    fac=100,
    # Change filter depending on filter used for observation
    filter_="F322W2", 
    # Change module depending on module used for observation
    module="A",
    # Is this needed?
    simYDisp=False,
    # Box around expected trace 
    returnRect=True,
    # Height 50 pixels as PSF modelled 50x50 pixels
    cross_disp_size=50,
    # Set Order desired to be computed for
    order=1,
    # Need some guidance on what this is !
    **patchkws,
):
    confFile = f"/Users/zx446701/OneDrive - The Open University/Ice_Age/Fengwu_Sims_2/GRISM_NIRCAM-master/V3/NIRCAM_{filter_}_mod{module}_R.conf"
    conf = grismconf.Config(confFile)
    # X and Y disp polynomials with 2 steps, the start [0] and end [1] of the trace
    dxs = conf.DISPX(f'+{order}', *pixels, np.array([0, 1]))
    # Keep in in case the JWST dispersion is curved and we need to trace the change in the curve
    dys = conf.DISPY(f'+{order}', *pixels, np.array([0, 1]))
    
    # Locating the centre of the trace in dispersion direction [0.5]
    centrePix_X = conf.DISPX(f'+{order}', *pixels, np.array([0.5]))
    
    # Locating Cross-dispersion centre of the trace
    centrePix_Y = conf.DISPY(f'+{order}', *pixels, np.array([0.5]))

    # Slight Rotation of tracebox due to slant in trace
    angle = np.rad2deg(np.arctan((dys[1]-dys[0])/(dxs[1]-dxs[0])))
    
    if returnRect:
#         mplplot.scatter(pixels[0]+centrePix[0], pixels[1],c='green')
        return mplpatches.Rectangle(
            # x0,y0 in bottom left of rectangle
            (pixels[0] + dxs[0], pixels[1] + dys[0] - (cross_disp_size // 2)),
            # width of rectangle 
            dxs[1] - dxs[0],
            # height of box (PSF width 50 pixels)
            cross_disp_size,
            # Slight Rotation of tracebox due to slant in trace
            angle=angle,
            **patchkws,
        )
    # Returns Central x and y of trace and dimensions of tracebox (height, width)
    return (pixels[0]+centrePix_X[0], pixels[1]+centrePix_Y[0]), (cross_disp_size, abs(dxs[1] - dxs[0]))



### Function to calculate the trace box of a source dispersed in the C direction

In [13]:
def computeTraceBoxC(
    # Pixels of source being traced (x0, y0)
    pixels,
    # Not sure this is needed
    fac=100,
    # Change filter depending on filter used for observation
    filter_="F322W2",
    # Change module depending on module used for observation
    module="A",
    # Is this needed?
    simYDisp=False,
    # Box around expected trace 
    returnRect=True,
    # Height 50 pixels as PSF modelled 50x50 pixels
    cross_disp_size=50,
    # Set Order desired to be computed for
    order=1,
    # Need some guidance on what this is !
    **patchkws,
):
    confFile = f"/Users/zx446701/OneDrive - The Open University/Ice_Age/Fengwu_Sims_2/GRISM_NIRCAM-master/V3/NIRCAM_{filter_}_mod{module}_C.conf"
    conf = grismconf.Config(confFile)
    # X and Y disp polynomials with 2 steps, the start and end of the trace
    dxs = conf.DISPX(f'+{order}', *pixels, np.array([0, 1]))
    # Keep in in case the JWST dispersion is curved and we need to trace the change in the curve
    dys = conf.DISPY(f'+{order}', *pixels, np.array([0, 1]))
    
    # Locating the centre of the trace in dispersion direction
    centrePix_Y = conf.DISPY(f'+{order}', *pixels, np.array([0.5]))
    
    # Locating the centre of the trace in CROSS-dispersion direction
    centrePix_X = conf.DISPX(f'+{order}', *pixels, np.array([0.5]))
    
    # Slight Rotation of tracebox due to slant in trace
    angle = np.rad2deg(np.arctan((dxs[1]-dxs[0])/(dys[1]-dys[0])))
    
    if returnRect:
#         mplplot.scatter(pixels[0]+centrePix[0], pixels[1],c='green')
        return mplpatches.Rectangle(
            # x0,y0 in bottom left of rectangle
            (pixels[0] + dxs[0] - cross_disp_size // 2, pixels[1] + dys[0]),
            # width of rectangle 
            cross_disp_size,
            # height of box (PSF width 50 pixels)
            dys[1] - dys[0],
            # Slight Rotation of tracebox due to slant in trace
            angle=angle,
            **patchkws,
        )
    # source Y pos + central Y pix of trace length, same but for X, PSF width (50 pix), trace length (~1365)
    return (pixels[1]+centrePix_Y[0], pixels[0]+centrePix_X[0]), (cross_disp_size, abs(dys[1] - dys[0]))


### Function to choose between R or C trace box required

In [14]:
def computeTraceBox(
    # Pixels of source being traced (x0, y0)
    pixels,
    # Not sure this is needed
    fac=100,
    # Change filter depending on filter used for observation
    filter_="F322W2",
    # Change module depending on module used for observation
    module="A",
    # Change Direction depending on disperser used for observation
    direction="R",
    # Is this needed?
    simYDisp=False,
    # Box around expected trace 
    returnRect=True,
    # Height 50 pixels as PSF modelled 50x50 pixels
    cross_disp_size=50,
    # Set Order desired to be computed for
    order=1,
    # Need some guidance on what this is !
    **patchkws,
):
    if direction == "R":
        return computeTraceBoxR(pixels=pixels,fac=fac,filter_=filter_,module=module,simYDisp=simYDisp,returnRect=returnRect,cross_disp_size=cross_disp_size,order=order,**patchkws)
    else:
        return computeTraceBoxC(pixels=pixels,fac=fac,filter_=filter_,module=module,simYDisp=simYDisp,returnRect=returnRect,cross_disp_size=cross_disp_size,order=order,**patchkws)

### Function returning Trace Box of 1st Order dispersions 

In [15]:
def compute1stOrderTraceBox(
    # Pixels of source being traced (x0, y0)
    pixels,
    # Not sure this is needed
    fac=100,
    # Change filter depending on filter used for observation
    filter_="F322W2",
    # Change module depending on module used for observation
    module="A",
    # Change Direction depending on disperser used for observation
    direction="R",
    # Is this needed?
    simYDisp=False,
    # Box around expected trace 
    returnRect=True,
    # Height 50 pixels as PSF modelled 50x50 pixels
    cross_disp_size=50,
    # Need some guidance on what this is !
    **patchkws,
):
    return computeTraceBox(pixels,fac,filter_,module,direction,simYDisp,returnRect,cross_disp_size,1,**patchkws)

### Function returning Trace Box of 2nd Order dispersions 

In [16]:
def compute2ndOrderTraceBox(
    # Pixels of source being traced (x0, y0)
    pixels,
    # Not sure this is needed
    fac=100,
    # Change filter depending on filter used for observation
    filter_="F322W2",
    # Change module depending on module used for observation
    module="A",
    # Change Direction depending on disperser used for observation
    direction="R",
    # Is this needed?
    simYDisp=False,
    # Box around expected trace 
    returnRect=True,
    # Height 50 pixels as PSF modelled 50x50 pixels
    cross_disp_size=50,
    # Need some guidance on what this is !
    **patchkws,
):
    if filter_ == "F322W2":
        return computeTraceBox(pixels,fac,filter_,module,direction,simYDisp,returnRect,cross_disp_size,2,**patchkws)
    else:
        print("You have made an error. 2nd Order spectra only occur in F332W2 frames. Please comment out patch.")

## Trim Offset Function

Calculates the maximum cross-dispersion offset for a trace in either the first or 2nd order to allow for traces of both orders to be included within the bounds of the in-frame sources dataframe.

Absolute values required to be taken as offsets can be minus numbers

In [20]:
def TrimOffsetFunc(conf,direction):
    if direction =="R":
        return np.amax([abs(conf._DISPY_data[f"+{o}"]) for o in range(1,3)])
    else:
        return np.amax([abs(conf._DISPX_data[f"+{o}"]) for o in range(1,3)])